In [1]:
#!rm W* -rf
#!rm w* -rf
#!rm s* -rf


In [2]:
!git clone --branch inference-tf-2.x https://github.com/steubk/White-box-Cartoonization.git

Cloning into 'White-box-Cartoonization'...
remote: Enumerating objects: 188, done.
remote: Total 188 (delta 0), reused 0 (delta 0), pack-reused 188
Receiving objects: 100% (188/188), 63.41 MiB | 9.13 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [3]:
!pip install --upgrade tf_slim
!pip install tensorflowjs
!mkdir saved_model_dir

Requirement already up-to-date: tf_slim in /usr/local/lib/python3.8/dist-packages (1.1.0)
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [4]:
import sys
sys.path.append('./White-box-Cartoonization/test_code')

#import matplotlib.pyplot as plt
import cartoonize
import os

#from PIL import Image
import network
import guided_filter

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [5]:
model_path = './White-box-Cartoonization/test_code/saved_models'
tf.reset_default_graph()
  
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    # Create placeholder for the input
    input_photo = tf.placeholder(tf.float32, [1, None, None, 3], name='input_photo')

    # Run the input placeholder through the generator, and then apply a 
    # filter to process the generator output
    network_out = network.unet_generator(input_photo)
    final_out = guided_filter.guided_filter(input_photo, network_out, r=1, eps=5e-3)
    final_out = tf.identity(final_out, name='final_output') # Create an identical filtering layer 

    # The above process is basically needed to construct the computation graph for the
    # current session

    # Get the generator variables and restore the pre-trained checkpoints in the 
    # current session
    all_vars = tf.trainable_variables()
    gene_vars = [var for var in all_vars if 'generator' in var.name]
    saver = tf.train.Saver(var_list=gene_vars)
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(model_path))

    # Export to SavedModel
    tf.saved_model.simple_save(
        sess,
        'saved_model_dir',
        inputs={input_photo.name: input_photo},
        outputs={final_out.name: final_out})

Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Restoring parameters from ./White-box-Cartoonization/test_code/saved_models/model-33999
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_model_dir/saved_model.pb


In [6]:
!tensorflowjs_converter --input_format=tf_saved_model --output_node_names="final_output" --saved_model_tags=serve saved_model_dir/ web_model_dir/
!tensorflowjs_converter --input_format=tf_saved_model --output_node_names="final_output" --saved_model_tags=serve saved_model_dir/ web_model_dir_int8/ --quantize_uint8=*

2020-10-23 17:19:57.401982: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-23 17:19:58.273873: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-23 17:19:58.287022: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-23 17:19:58.287532: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce GTX 1660 computeCapability: 7.5
coreClock: 1.785GHz coreCount: 22 deviceMemorySize: 5.74GiB deviceMemoryBandwidth: 178.86GiB/s
2020-10-23 17:19:58.287548: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-23 17:19:58.288490: I tensorflow/stream_executor/platform